In [10]:
import numpy as np
import time

## Utils

In [152]:
def int2bin(x: int, size: int):
    return bin(x)[2:].zfill(size)


def float2int(x: float, n: int):
    return int(x * pow(2, n))

## Bit Stream

In [25]:
class BitStream:
    def __init__(self):
        self.stream = 0  # The content of the stream
        self.size = 0  # The current size of the stream

    def add_bits(self, bits: int, bit_count: int):
        """Add bits to the stream.

        Args:
            bits (int): The integer containing the bits to add (e.g.  0b101).
            bit_count (int): The number of bits to add from the 'bits' integer (e.g. 3).
        """
        self.stream = (self.stream << bit_count) | (bits & ((1 << bit_count) - 1))
        self.size += bit_count

    def pop_bits(self, bit_count: int) -> int:
        """Read qnd remove a certain number of bits from the stream.

        Args:
            bit_count (int): The number of bits to read.

        Returns:
            int: The extracted bits as an integer.
        """
        if bit_count > self.size:
            raise ValueError("Not enough bits in the stream to read.")

        # Extract the bits from the most significant part of the stream
        result = self.stream >> (self.size - bit_count)
        # Remove the bits we just read
        self.stream &= (1 << (self.size - bit_count)) - 1
        self.size -= bit_count

        return result

    def read(self):
        """Read the content of the stream as a binary int

        Returns:
            int: the content of the stream as an integer
        """
        return self.stream

    def __str__(self) -> str:
        """Return the bit stream as a binary string for easy visualization."""
        return bin(self.stream)[2:].zfill(self.size)

In [33]:
bitStream = BitStream()
bitStream.add_bits(0b00010, 5)
print(bin(bitStream.stream)[2:].zfill(5))
bitStream.pop_bits(2)
print(bin(bitStream.stream)[2:].zfill(2))

00010
10


## LFSR

In [178]:
main_seed = int(np.random.random() * (pow(2, 128) - 1))
main_taps = [0, 1, 2, 7]

In [134]:
class LFSR:
    def __init__(self, seed: int, taps: list[int], size: int):
        self.seed = seed
        self.taps = taps
        self.size = size
        self.register = seed

    def step(self):
        feedback = 0
        for tap in self.taps:
            feedback ^= (self.register >> tap) & 1

        # Shift the register to the right and set the most significant bit to the feedback bit
        self.register = (self.register >> 1) | (feedback << (self.size - 1))

        return self.register

# B2S and S2B convertor

In [186]:
class Binary2Stochastic:
    def __init__(self, lfsr_seed: int, lfsr_taps: int, lfsr_size: int, output_len: int):
        self.lfsr = LFSR(seed=lfsr_seed, taps=lfsr_taps, size=lfsr_size)
        self.n = lfsr_size
        self.output_len = output_len

    def convert(self, x: float):
        assert 0 < x < 1, "Input needs to be in range [0,1]"
        x_as_binary = float2int(x, self.n)
        result = 0
        for _ in range(self.output_len):
            rng = self.lfsr.step()
            result = (result << 1) | (x_as_binary < rng)
        return result

In [190]:
stoch_len = 1000000
b2s = Binary2Stochastic(main_seed, main_taps, 128, stoch_len)
sn = b2s.convert(0.48)
res = 0
for el in int2bin(sn, stoch_len):
    res += int(el)
print(res / stoch_len)

0.520058
